In [13]:
import jax
import jax.numpy as jnp
from annealed_flow_transport import train
from configs import lgcp_pines
from configs import funnel
from ml_collections import ConfigDict
from flax import linen as nn
import chex
from new_aft_stuff.utils import smc_utils as su

In [14]:
key = jax.random.PRNGKey(0)
key, key_ = jax.random.split(key)

In [40]:
jnp.ones(1)

Array([1.], dtype=float32)

In [5]:
config = ConfigDict()

config.seed = 5
config.batch_size = 2000
# config.sample_shape = (32*32,)
config.sample_shape = (10,)
config.report_step = 1
config.num_temps = 10
config.resample_threshold = 0.3
config.stopping_criterion = 'time'
config.use_resampling = True
config.use_markov = True
config.use_path_gradient = False
config.algo = 'craft'
config.vi_iters = 1000
config.vi_estimator = 'importance'
config.vi_report_step = 100
config.craft_num_iters = 200
config.craft_batch_size = 2000

optimization_config = ConfigDict()
# optimization_config.free_energy_iters = 500 (this should be aft_num_iters)
# optimization_config.aft_step_size = 1e-2
optimization_config.craft_step_size = 1e-3
optimization_config.vi_step_size = 1e-3
config.optimization_config = optimization_config

initial_config = ConfigDict()
initial_config.density = 'MultivariateNormalDistribution'
initial_config.shared_mean = 0.
initial_config.diagonal_cov = 1.
config.initial_config = initial_config

final_config = ConfigDict()
# final_config.density = 'LogGaussianCoxPines'
final_config.density = 'FunnelDistribution'
# final_config.use_whitened = False
# final_config.file_path = 'data/finpines.csv' ################################################
config.final_config = final_config

initial_sampler_config = ConfigDict()
initial_sampler_config.initial_sampler = 'MultivariateNormalDistribution'
config.initial_sampler_config = initial_sampler_config

flow_config = ConfigDict()
flow_config.type = 'DiagonalAffine'
flow_config.sample_shape = config.sample_shape
config.flow_config = flow_config

mcmc_config = ConfigDict()
hmc_step_config = ConfigDict()
hmc_step_config.step_times = [0., 0.25, 0.5, 1.]
hmc_step_config.step_sizes = [0.3, 0.3, 0.2, 0.2]

mcmc_config.hmc_step_config = hmc_step_config
mcmc_config.hmc_steps_per_iter = 1
mcmc_config.rwm_steps_per_iter = 0
mcmc_config.hmc_num_leapfrog_steps = 10
config.mcmc_config = mcmc_config

In [6]:
res = train.run_experiment(config)
res

AlgoResultsTuple(test_samples=Array([[-0.72677284, -0.26233655, -0.27031282, ..., -0.38044098,
        -0.07710744,  0.19597872],
       [-2.3037531 ,  0.61401784,  0.3610436 , ...,  0.29342684,
         0.21329764, -0.08818472],
       [-0.5597671 , -1.1436883 , -1.2696712 , ..., -0.40046382,
         1.8555237 ,  1.2299283 ],
       ...,
       [ 0.33880764, -0.95755666, -1.4909831 , ..., -0.58816355,
        -1.1841233 , -0.4133686 ],
       [-0.9067402 ,  0.09210262, -0.01061814, ..., -0.4997457 ,
        -0.17286083, -0.1135186 ],
       [-2.5126436 ,  0.23274058, -0.3197874 , ...,  0.6104518 ,
         0.12019639,  0.21252064]], dtype=float32), test_log_weights=Array([-7.9212103, -8.097366 , -8.137023 , ..., -7.7042093, -7.8161287,
       -7.5426273], dtype=float32), log_normalizer_estimate=Array(-0.24894142, dtype=float32), delta_time=19.98450493812561, initial_time_diff=2.1193394660949707)